In [1]:
# Imports iniciales
%matplotlib inline
from common import *
from ann import *
from sklearn.model_selection import KFold

In [2]:
# Ejercicio 1

epochs=25 # numero de epocas que entrena cada vez
eta=0.01 # learning rate
alfa=0.9 # momentum
N2=60  # neuronas en la capa oculta
evals = 1000 # evalauciones del entrenamiento

df_spirals_600 = generate_spirals(600)
kf = KFold(n_splits = 5, shuffle = True)
result = next(kf.split(df_spirals_600), None)
df_train_spirals = df_spirals_600.iloc[result[0]]
df_val_spirals = df_spirals_600.iloc[result[1]]

df_test_spirals = generate_spirals(2000)

X_train, y_train = df_train_spirals[[0, 1]], df_train_spirals['Class']
X_val, y_val = df_val_spirals[[0, 1]], df_val_spirals['Class']
X_test, y_test = df_test_spirals[[0, 1]], df_test_spirals['Class']


for N2 in [2, 10, 20, 40]:
  classif = create_net_classifier(N2, eta, alfa, epochs)
  best_net, e_train, e_val, e_test = train_net(classif, evals, X_train, y_train, X_val, y_val, X_test, y_test)
